# Part 1 of 3

In [61]:
import numpy as np
import pandas as pd
import requests
!pip install bs4
from bs4 import BeautifulSoup
import folium
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from sklearn.cluster import KMeans

In [2]:
!pip install geopy
from geopy.geocoders import Nominatim 

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
website = requests.get(url).text

In [5]:
soup = BeautifulSoup(website,'html5lib')

In [6]:
table = soup.find('table')

In [7]:
#Get all rows from the table
for row in table.find_all('tr'): # in html table row is represented by the tag <tr>
    # Get all columns in each row.
    cols = row.find_all('td') # in html a column is represented by the tag <td>
    #color_name = cols[2].string # store the value in column 3 as color_name
    #color_code = cols[3].string # store the value in column 4 as color_code
    #print("{}--->{}".format(color_name,color_code))

In [8]:
len(table)

2

In [9]:
table_rows=table.find_all('tr')

In [10]:
#soup.table()[50]
len(table_rows)

20

In [11]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [12]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [14]:
count_row = df.shape[0]
count_column = df.shape[1]
print('count_column= ', count_column)
print('count_row= ', count_row)
df.shape

count_column=  3
count_row=  103


(103, 3)

# Part 2 of 3

In [15]:
df_po = pd.read_csv('Geospatial_Coordinates.csv')

In [16]:
df_po.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_po.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Postal Code  103 non-null    object 
 1   Latitude     103 non-null    float64
 2   Longitude    103 non-null    float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [18]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [19]:
df_po.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [20]:
df_po.rename({'Postal Code':'PostalCode'}, axis=1, inplace = True)

In [21]:
df_po.index
df_po.columns
df_po[["PostalCode"]]
#df[["PostalCode"]]
#new_df = df.merge(df2[["ID","grade"]], on="ID", how="left")

,PostalCode
0,M1B
1,M1C
2,M1E
3,M1G
4,M1H
...,...
98,M9N
99,M9P
100,M9R
101,M9V


In [22]:
df = pd.merge(df, df_po,on='PostalCode', how='inner')

In [23]:
df.loc[df['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [24]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
